# СЕРВИС ФОРМИРОВАНИЯ ЗАДАЧ ДЛЯ МОСКВИЧЕЙ ПО КОНТРОЛЮ РАБОТ ПОДРЯДЧИКОВ В СФЕРЕ ГОРОДСКОГО БЛАГОУСТРОЙСТВА
## Задача
**Разработка сервиса, который с помощью распознавания документации проектов городского благоустройства сформирует для москвичей задания по мониторингу выполненных подрядчиками работ**


In [1]:
import os
import pandas as pd
import numpy as np

#Для работы с Excel
import xlrd, xlwt
from xlsxwriter.utility import xl_rowcol_to_cell
import openpyxl

pd.set_option('display.max_columns', 500)

## Обработаем файл Шаблон ТЗ

In [2]:
pattern_tz = pd.read_excel('D:\IT\Hackatones\Lider_IT_22\ТЗ\Шаблон ТЗ.xlsx') 

pattern_tz.info()
display(pattern_tz.head())
pattern_tz.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165405 entries, 0 to 165404
Data columns (total 4 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Наименование шаблона ТЗ  38212 non-null   object
 1   КПГЗ                     165405 non-null  object
 2   ID                       165405 non-null  int64 
 3   Наименование СПГЗ        165405 non-null  object
dtypes: int64(1), object(3)
memory usage: 5.0+ MB


,Наименование шаблона ТЗ,КПГЗ,ID,Наименование СПГЗ
0,NaN,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕ...,40209538,Поверка грузов к грузопоршневому манометру
1,NaN,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕ...,40209531,Поверка грузопоршневого манометра
2,NaN,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕ...,40209536,Поверка измерителя параметров электроизоляции
3,NaN,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕ...,40209532,Поверка измерителя сопротивления заземления
4,NaN,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕ...,40209559,Поверка индукционного трансформатора тока


,Наименование шаблона ТЗ,КПГЗ,ID,Наименование СПГЗ
165400,NaN,"01.02.10.43.765.01 РН МОЧИ ИВД, НАБОР, КОЛОРИМ...",27263995,"рН мочи ИВД, набор, колориметрическая тест-пол..."
165401,NaN,"01.02.10.43.765.01 РН МОЧИ ИВД, НАБОР, КОЛОРИМ...",27263996,"рН мочи ИВД, набор, колориметрическая тест-пол..."
165402,NaN,"01.02.10.43.765.01 РН МОЧИ ИВД, НАБОР, КОЛОРИМ...",27263960,"рН мочи ИВД, набор, колориметрическая тест-пол..."
165403,NaN,"01.02.10.43.765.01 РН МОЧИ ИВД, НАБОР, КОЛОРИМ...",27263992,"рН мочи ИВД, набор, колориметрическая тест-пол..."
165404,NaN,"01.11.04.04 КАНАТЫ, ШНУРЫ, ТРОСЫ",3372533,"шпагат (веревка), шт"


In [4]:
pattern_tz = (
    pattern_tz
    .rename(columns={'Наименование шаблона ТЗ': 'pattern',
                                    'КПГЗ': 'kpgz',
                                    'ID': 'id',
                                    'Наименование СПГЗ': 'spgz'})
    .replace(np.nan, '')
)
pattern_tz['id'] = pattern_tz['id'].astype('int')
pattern_tz

,pattern,kpgz,id,spgz
0,,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕ...,40209538,Поверка грузов к грузопоршневому манометру
1,,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕ...,40209531,Поверка грузопоршневого манометра
2,,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕ...,40209536,Поверка измерителя параметров электроизоляции
3,,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕ...,40209532,Поверка измерителя сопротивления заземления
4,,03.30.06.04 МЕТРОЛОГИЧЕСКОЕ ОБСЛУЖИВАНИЕ НЕ МЕ...,40209559,Поверка индукционного трансформатора тока
...,...,...,...,...
165400,,"01.02.10.43.765.01 РН МОЧИ ИВД, НАБОР, КОЛОРИМ...",27263995,"рН мочи ИВД, набор, колориметрическая тест-пол..."
165401,,"01.02.10.43.765.01 РН МОЧИ ИВД, НАБОР, КОЛОРИМ...",27263996,"рН мочи ИВД, набор, колориметрическая тест-пол..."
165402,,"01.02.10.43.765.01 РН МОЧИ ИВД, НАБОР, КОЛОРИМ...",27263960,"рН мочи ИВД, набор, колориметрическая тест-пол..."
165403,,"01.02.10.43.765.01 РН МОЧИ ИВД, НАБОР, КОЛОРИМ...",27263992,"рН мочи ИВД, набор, колориметрическая тест-пол..."


In [5]:
# Сохраним очищенный справочник
pattern_tz.to_excel('pattern_tz.xlsx', index=False)